In [1]:
import numpy as np
import time

# DNN functions in numpy

In [2]:
def relu(x):
    return np.maximum(x, 0)

def conv2d(x, w, b):
    # x: [C_in, H, W]
    # w: [C_out, C_in, K, K]
    # b: [C_out]

    C_out, C_in, K, _ = w.shape
    _, H, W = x.shape

    H_out = H - K + 1
    W_out = W - K + 1

    x_win = np.stack(
        [x[:, i:i+H_out, j:j+W_out]
         for i in range(K)
         for j in range(K)],
        axis=-1
    ).reshape(C_in, H_out, W_out, K, K)

    y = np.sum(
        x_win[None] * w[:, :, None, None],
        axis=(1, 4, 5)
    )

    y += b[:, None, None]
    return y

def maxpool2x2(x):
    # x: [C, H, W]
    C, H, W = x.shape
    return x.reshape(C, H//2, 2, W//2, 2).max(axis=(2, 4))

def linear(x, w, b):
    # x: [N]
    # w: [M, N]
    # b: [M]
    return w @ x + b


# DNN Model

In [3]:
def cnn_inference(
    x,
    weights
):
    # x: [1, 28, 28]
    x = x.astype(np.float32)

    # Conv1
    x = conv2d(x, weights["conv1_w"], weights["conv1_b"])
    x = relu(x)

    # Conv2
    x = conv2d(x, weights["conv2_w"], weights["conv2_b"])
    x = relu(x)

    # Pool
    x = maxpool2x2(x)

    # Conv3
    x = conv2d(x, weights["conv3_w"], weights["conv3_b"])
    x = relu(x)

    # Pool
    x = maxpool2x2(x)

    # Flatten
    x = x.reshape(-1)          # [4*5*5 = 100]

    # FC1
    x = linear(x, weights["fc1_w"], weights["fc1_b"])
    x = relu(x)

    # FC2
    x = linear(x, weights["fc2_w"], weights["fc2_b"])

    return x


# Load model weights and bias from the saved file


In [4]:
data = np.load("model_weights.npz")

weights = {
    "conv1_w": data["conv1_w"],
    "conv1_b": data["conv1_b"],
    "conv2_w": data["conv2_w"],
    "conv2_b": data["conv2_b"],
    "conv3_w": data["conv3_w"],
    "conv3_b": data["conv3_b"],
    "fc1_w": data["fc1_w"],
    "fc1_b": data["fc1_b"],
    "fc2_w": data["fc2_w"],
    "fc2_b": data["fc2_b"],
}


# Load dataset from the saved MNIST dataset

### The dataset contains 10000 samples of handwritten digits
### Input Data shape: 1x28x28
### The input data is normalized to be between 0 and 1
### target_labels contains the correct label.

In [5]:
data = np.load("dataset.npz")

input_data = data["input_data"]
target_labels = data["target_labels"]


# Model Evaluation.
### We load one sample from the dataset, pass it to the model. 
### We Also show the input as image 

In [6]:
# input image
import matplotlib.pyplot as plt

index = 25
x = input_data[index]  # shape [1, 28, 28]
logits = cnn_inference(x, weights)
pred = np.argmax(logits)


print("Predicted class:", pred)
print("Actual class   :", target_labels[index])

np.set_printoptions(linewidth=250)
print("\n\nClass | Logits")
print("------|----------------------")

for t in np.arange(10):
    print(f'  {t}   |  {logits[t]}')
print("------|----------------------")
x_uint8 = (x * 255.0).clip(0, 255).astype(np.uint8)
x_uint8 = 255-x_uint8
img = x_uint8[0]
plt.imshow(img, cmap="gray")
plt.title("Input Image")
plt.show()

Predicted class: 0
Actual class   : 0


Class | Logits
------|----------------------
  0   |  6.846710205078125
  1   |  -15.208741188049316
  2   |  1.8967386484146118
  3   |  -13.56126594543457
  4   |  -10.309934616088867
  5   |  -6.790517807006836
  6   |  0.2771644592285156
  7   |  -5.117846488952637
  8   |  -3.097658634185791
  9   |  -8.148666381835938
------|----------------------


# To get the Accuracy of the model.


In [7]:
correct = 0
total = len(input_data[0:200])  ## we only use 200 samples to save time.
total_time=0

for i in range(total):
    x = input_data[i]        # shape [1, 28, 28]
    target = target_labels[i]
    
    ps_start=time.time()

    logits = cnn_inference(x, weights)
    
    ps_stop=time.time()
    pred = np.argmax(logits)

    if pred == target:
        correct += 1
        
    total_time += ps_stop-ps_start

accuracy = 100.0 * correct / total
print("Accuracy:", accuracy, "%")

av_PS_time = total_time/total
print(f'Average PS Time: {total_time/total}')


Accuracy: 99.0 %
Average PS Time: 0.013366501331329345


# Hardware Testing

In [8]:
from pynq import Overlay
from pynq import allocate
ol = Overlay("design_vitis_new.bit")
dma=ol.axi_dma_0
dma_send = dma.sendchannel
dma_recv = dma.recvchannel

#### Baseline Software Implementation

In [9]:
index = 10
x = input_data[index]  # shape [1, 28, 28]
input_image = x.reshape(-1)

ps_start=time.time()
logits = cnn_inference(x, weights)
ps_stop=time.time()

pred_sw = np.argmax(logits)

ps_time=ps_stop-ps_start
print(f'PS Time : {ps_time}')

PS Time : 0.02448725700378418


#### Hardware implementation

In [10]:
index = 10
x = input_data[index]  # shape [1, 28, 28]


input_buffer = allocate(shape=(784,), dtype=np.float32)
output_buffer = allocate(shape=(10,), dtype=np.float32)
np.copyto(input_buffer,input_image)

pl_start=time.time()
dma_send.transfer(input_buffer)
dma_recv.transfer(output_buffer)
dma_send.wait()
dma_recv.wait()

pl_stop=time.time()

pred_hw = np.argmax(output_buffer)


pl_time=pl_stop-pl_start
print(f'PL Time : {pl_time}')

PL Time : 0.005547761917114258


#### Run hardware 300 times 

In [11]:
correct = 0
total = len(input_data[0:300])  ## we only use 200 samples to save time.
total_time=0.0

for i in range(total):
    x = input_data[i]        # shape [1, 28, 28]
    target = target_labels[i]

    input_image = x.reshape(-1)
    np.copyto(input_buffer,input_image)

    pl_start=time.time()
    dma_send.transfer(input_buffer)
    dma_recv.transfer(output_buffer)
    dma_send.wait()
    dma_recv.wait()

    pl_stop=time.time()
        
    pred_hw = np.argmax(output_buffer)


    if pred_hw == target:
        correct += 1
        
    total_time += pl_stop-pl_start

accuracy = 100.0 * correct / total
print("Accuracy:", accuracy, "%")

av_PL_time = total_time/total
print(f'Average PL Time: {total_time/total}')


Accuracy: 98.66666666666667 %
Average PL Time: 0.0049030685424804685


In [12]:
Speedup = av_PS_time/av_PL_time
print(f'Average Speedup using Hardware = {Speedup}')

Average Speedup using Hardware = 2.7261502088990204
